In [1]:
import datetime
from pyspark import SparkConf, SparkContext, HiveContext
sc.stop()
conf = SparkConf().set("spark.executor.instances", 32).set("spark.driver.maxResultSize", "32g")
sc = SparkContext(conf=conf)
#sc.setCheckpointDir('/user/kposminin/checkpointdir/')

In [2]:
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import BinaryClassificationMetrics
import sklearn
import sklearn.ensemble
import numpy as np

# Load and parse the data file.
train = sc.textFile("/user/kposminin/la_20160817") \
  .filter(lambda s: (s[0] == '1') or (hash(s) % 20000 == 1)) \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .collect()
test = sc.textFile("/user/kposminin/la_20160824") \
  .filter(lambda s: (s[0] == '1') or (hash(s) % 20000 == 0)) \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .collect()

test_rdd = sc.textFile("/user/kposminin/la_20160824") \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .filter(lambda r: len(r) == 29)

test_rdd2 = sc.textFile("/user/kposminin/la_20160818") \
  .map(lambda r:r.split('\t')) \
  .map(lambda r:[int(e) for e in r]) \
  .filter(lambda r: len(r) == 29)

In [3]:
modelGBT = sklearn.ensemble.GradientBoostingClassifier(n_estimators=80, learning_rate=0.1,
       max_depth=6, random_state=0).fit(X = [e[1:] for e in train], y = [e[0] for e in train])

In [6]:
def test_model_on_rdd(model,rdd):
    res = rdd.map(lambda r:(r[0],int(10**4 * model.predict_proba(r[1:])[0][1]),r[1])).collect()
    AUCROC['smax'] = sklearn.metrics.roc_auc_score(y_true = [e[0] for e in res], y_score = [e[2] for e in res])
    AUCROC['model'] = sklearn.metrics.roc_auc_score(y_true = [e[0] for e in res], y_score = [e[1] for e in res])
    return AUCROC


In [7]:
print(test_model_on_rdd(modelGBT,test_rdd))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 3 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:806)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:804)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:804)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1658)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1581)
	at org.apache.spark.SparkContext$$anonfun$stop$7.apply$mcV$sp(SparkContext.scala:1731)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1229)
	at org.apache.spark.SparkContext.stop(SparkC

In [ ]:
print(test_model_on_rdd(modelGBT,test_rdd2))